In [1]:
import os
import numpy as np
import pandas as pd
from analysis import *

import random

from sklearn.cluster import DBSCAN

from pyclustering.cluster import cluster_visualizer_multidim
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.kmeans import *
from pyclustering.cluster.kmedians import *
from pyclustering.cluster.optics import *
from pyclustering.cluster.rock import *
from pyclustering.cluster.dbscan import *
from pyclustering.cluster.elbow import elbow

from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import numpy as np

import seaborn as sns

In [2]:
sdss_df = import_sdss ("%s/Databases/SDSSMOC4/data/sdssmocadr4.tab" % os.getcwd())

# Drop information not relevant to this problem, along with the SMOC_ID, which just indicates
# an SDSS observation; one asteroid can have multiple SMOC_ID, so it isn't a useful identifier.
sdss_df.drop(labels=['OBJ_ID_RUN', 'OBJ_ID_COL', 'OBJ_ID_FIELD',
                     'OBJ_ID_OBJ', 'ROWC', 'COLC', 'JD_ZERO', 'RA',
                     'DEC', 'LAMBDA', 'BETA', 'PHI', 'VMU', 'VMU_ERROR',
                     'VNU', 'VNU_ERROR', 'VLAMBDA', 'VBETA', 'IDFLAG',
                     'RA_COMPUTED', 'DEC_COMPUTED', 'V_MAG_COMPUTED',
                     'R_DIST', 'G_DIST', 'OSC_CAT_ID', 'ARC',
                     'EPOCH_OSC', 'A_OSC', 'E_OSC', 'I_OSC', 'LON_OSC',
                     'AP_OSC', 'M_OSC', 'PROP_CAT_ID', 'A_PROP',
                     'E_PROP', 'SIN_I_PROP', 'V_MAG', 'B_MAG', 'H', 'G',
                     'A_MAG', 'A_ERR', 'SMOC_ID', 'PHASE', 'D_COUNTER', 
                     'TOTAL_D_COUNT'],
                     axis=1, inplace=True)

In [3]:
tax_df = import_tax("%s/Databases/SDSSTaxonomy/data/sdsstax_ast_table.tab" % os.getcwd())
tax_df.drop(labels=['AST_NAME', 'SCORE', 'NCLASS', 'METHOD', 'BAD',
                    'SEQUENCE', 'PROPER_SEMIMAJOR_AXIS',
                    'PROPER_ECCENTRICITY',
                    'SINE_OF_PROPER_INCLINATION',
                    'OSC_SEMIMAJOR_AXIS', 'OSC_ECCENTRICITY',
                    'OSC_INCLINATION'],
                     axis=1, inplace=True)

In [4]:
wave_mags = ["U_MAG", "G_MAG", "R_MAG", "I_MAG", "Z_MAG"]
wave_errs = ["U_ERR", "G_ERR", "R_ERR", "I_ERR", "Z_ERR"]

# Ensure data integrity. Every observation MUST have a U, G, R, I, and Z value, and associated error.
sdss_df.dropna(subset=wave_mags + wave_errs, inplace=True)

In [5]:
merged = merge(sdss_df, tax_df, ["PROV_ID", "AST_NUMBER"])
merged

,U_MAG,U_ERR,G_MAG,G_ERR,R_MAG,R_ERR,I_MAG,I_ERR,Z_MAG,Z_ERR,AST_NUMBER,PROV_ID,CLASSIFICATION,MOID,H
251468,21.81,0.20,20.32,0.04,19.77,0.02,19.56,0.03,19.44,0.14,62869,2000 UO84,XD,s0584d,14.600000
251469,22.04,0.23,20.36,0.03,19.82,0.03,19.66,0.03,19.44,0.11,62869,2000 UO84,XD,s0584d,14.600000
251471,19.69,0.03,17.91,0.02,17.32,0.01,17.10,0.02,17.05,0.03,5212,1989 SS,L,s05871,11.600000
251473,21.83,0.16,19.82,0.02,19.16,0.02,18.94,0.02,19.02,0.06,35549,1998 FT108,S,s05869,14.200000
251474,22.40,0.24,20.40,0.02,19.73,0.02,19.47,0.03,19.76,0.10,35549,1998 FT108,S,s05869,14.200000
251476,21.96,0.17,20.30,0.03,19.95,0.03,19.73,0.03,19.71,0.10,148456,2000 YQ87,CX,sf3728,15.000000
251477,21.33,0.14,20.00,0.02,19.49,0.02,19.34,0.02,19.33,0.05,148456,2000 YQ87,CX,sf3728,15.000000
251482,21.54,0.12,19.66,0.02,19.09,0.02,18.96,0.03,18.85,0.05,39737,1997 AE1,C,s058d2,14.600000
251483,20.74,0.06,18.96,0.02,18.29,0.01,18.05,0.02,18.12,0.03,66569,1999 RM145,S,s058d4,15.600000
251484,20.75,0.19,18.99,0.02,18.32,0.01,18.13,0.02,18.19,0.04,66569,1999 RM145,S,s058d4,15.600000


In [9]:
training_data = sdss_df[wave_mags].values

In [ ]:
# Visualizing 5-D mix data using bubble charts
# leveraging the concepts of hue, size and depth

plot_5d(sdss_df["U_MAG"], sdss_df["G_MAG"], sdss_df["R_MAG"], sdss_df["I_MAG"], sdss_df["Z_MAG"],
        *wave_mags, "U_MAG - G_MAG - R_MAG - I_MAG - Z_MAG")

plot_5d(sdss_df["Z_MAG"], sdss_df["I_MAG"], sdss_df["R_MAG"], sdss_df["G_MAG"], sdss_df["U_MAG"],
        *(wave_mags[::-1]), "Z_MAG - I_MAG - R_MAG - G_MAG - U_MAG")

In [ ]:
plot_5d(sdss_df["G_MAG"], sdss_df["Z_MAG"], sdss_df["I_MAG"], sdss_df["U_MAG"], sdss_df["R_MAG"],
        *["G_MAG", "Z_MAG", "I_MAG", "U_MAG", "R_MAG"],
        "G_MAG - Z_MAG - I_MAG - U_MAG - R_MAG")

## Elbow Method
Manual computation of clusters, in the form of the equation

$$
Elbow_{k} = \frac{\left ( y_{0} - y_{1} \right )x_{k} + \left ( x_{1} - x_{0} \right )y_{k} + \left ( x_{0}y_{1} - x_{1}y_{0} \right )}{\sqrt{\left ( x_{1} - x_{0} \right )^{2} + \left ( y_{1} - y_{0} \right )^{2}}}
$$

given a kmin-point ($x_0$, $y_0$) the kmax-point ($x_1$, $y_1$), the value $x_k$, the amount of clusters, and $y_k$, the within-cluster error.

In [ ]:
kmin, kmax = 1, 20
elbow_instance = elbow(training_data, kmin, kmax)

In [ ]:
# process input data and obtain results of analysis
elbow_instance.process()
amount_clusters = elbow_instance.get_amount()

In [ ]:
amount_clusters

In [ ]:
# perform cluster analysis using K-Means algorithm
centers = kmeans_plusplus_initializer(training_data, amount_clusters).initialize()
kmeans_instance = kmeans(training_data, centers)
kmeans_instance.process()

In [ ]:
# obtain clustering results and visualize them
elbow_clusters = kmeans_instance.get_clusters()
elbow_centers = kmeans_instance.get_centers()

In [ ]:
draw_5d_clusters(sdss_df["G_MAG"], sdss_df["Z_MAG"], sdss_df["I_MAG"], sdss_df["U_MAG"], sdss_df["R_MAG"],
                 clusters, "G_MAG", "Z_MAG", "I_MAG", "U_MAG", "R_MAG", "G_MAG - Z_MAG - I_MAG - U_MAG - R_MAG")

In [ ]:
# Now, compare against Carvano.
elbow_classes = sdss_df.copy()

apply_classification(elbow_classes, elbow_clusters, "ELBOW_CLASSIFICATION")

merged_elbow_classes = merge(elbow_classes, tax_df, ["PROV_ID", "AST_NUMBER"])

In [ ]:
merged_elbow_classes

In [ ]:
res = compare_results(merged_elbow_classes, "ELBOW_CLASSIFICATION", "CLASSIFICATION")

for class_type, occurs in res:
    # Sort the dict for plotting
    sorted_keys = sorted(occurs)

    fig = plt.figure(figsize=(12, 5))
    fig.suptitle(class_type, fontsize=14)
    plt.bar(sorted_keys, [occurs[key] for key in sorted_keys])

In [ ]:
# Code to subdivide the clusters; takes the existing clusters and re-computes the elbow inside each cluster.
# Does not handle the data well at all.
cluster_list = []

for cluster_group in clusters:
    curr_cluster = [training_data[i] for i in cluster_group]
    elbow_instance = elbow(curr_cluster, kmin, kmax)
    elbow_instance.process()
    amount_clusters = elbow_instance.get_amount()
    print(amount_clusters)
    
    centers = kmeans_plusplus_initializer(training_data, amount_clusters).initialize()
    kmeans_instance = kmeans(training_data, centers)
    kmeans_instance.process()
    cluster_list.extend(kmeans_instance.get_clusters())

In [ ]:
draw_5d_clusters(sdss_df["G_MAG"], sdss_df["Z_MAG"], sdss_df["I_MAG"], sdss_df["U_MAG"], sdss_df["R_MAG"],
                 cluster_list, "G_MAG", "Z_MAG", "I_MAG", "U_MAG", "R_MAG", "G_MAG - Z_MAG - I_MAG - U_MAG - R_MAG")

## K-Means++
Now, we do a graphical analysis to determine the number of clusters.

In [ ]:
# Use k-means++ to guess centers
run_bound = 25
run_max = 10

all_args = [(training_data, i) for i in range(1, run_bound)]

with Pool(8) as p:
    all_output = [p.starmap(k_means_pp, all_args) for i in range(run_max)]

In [ ]:
fig = plt.figure(figsize=(14,10))
for i in range(run_max):
    # Get the full resample i's array of errors.
    err = [all_output[i][j][1] for j in range(run_bound - 1)]
    plt.plot(range(len(err) + 1), [None,] + err)
    
ax = fig.gca()
ax.set_xticks(numpy.arange(0, 25, 1))
plt.title("K-Means++ Cost vs. Clusters")
plt.ylabel("Sum of Squared Error")
plt.xlabel("Number of Clusters")
plt.grid()
plt.show()

## DBSCAN

In [10]:
data_sample = random.sample(training_data, 500)

TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [16]:
# Create OPTICS algorithm for cluster analysis
dbscan_instance = DBSCAN(eps=10, min_samples=1).fit(data_sample[:1000])
#dbscan_instance.labels_
np.unique(dbscan_instance.labels_)
np.savetxt("test.txt", dbscan_instance.labels_, fmt='%d')

In [ ]:
# Obtain results of clustering
clusters = dbscan_instance.get_clusters();
noise = dbscan_instance.get_noise();

In [ ]:
merged = merged[wave_mags + ["CLASSIFICATION", "ML_CLASS"]].dropna()

In [23]:
arr_size = len(sdss_df) - 1

index_dict = {}
while len(index_dict.keys()) < 5000:
    index_dict[random.randint(0, arr_size)] = 0

In [24]:
index_dict.keys()

dict_keys([338906, 470689, 159633, 252309, 435753, 17691, 293902, 34609, 16844, 210344, 191943, 181601, 232563, 249871, 115637, 104221, 339820, 113878, 255079, 339851, 69044, 38597, 365617, 37424, 198813, 91197, 268059, 357841, 352886, 221506, 384239, 319448, 65267, 102319, 32716, 135078, 207479, 117863, 303617, 37680, 397588, 176040, 3634, 242046, 391800, 340857, 38412, 92705, 67051, 209991, 138824, 32093, 360114, 406605, 217632, 166861, 284474, 190817, 375395, 355872, 19645, 436320, 142496, 216411, 66870, 131190, 317317, 289141, 384428, 368229, 382720, 456451, 15228, 325411, 184492, 15369, 186773, 58023, 439265, 332166, 133806, 97098, 273180, 66483, 138828, 98597, 80216, 231366, 247974, 266912, 341513, 173896, 38385, 196225, 68779, 150734, 462370, 85605, 111046, 147874, 206509, 43025, 382906, 366616, 214827, 373182, 322326, 12799, 92371, 28785, 372733, 457763, 190147, 278016, 303923, 213124, 290408, 260173, 31426, 26682, 205055, 44126, 159731, 220468, 83518, 80257, 353475, 316955, 46

In [22]:
arr_size = len(sdss_df)
arr_size

471569

In [26]:
np.array([training_data[idx] for idx in index_dict.keys()])

array([[23.95, 21.4 , 20.94, 20.62, 21.19],
       [20.95, 19.27, 18.65, 18.5 , 18.47],
       [22.08, 20.34, 19.74, 19.57, 19.61],
       ...,
       [22.98, 21.46, 20.89, 20.75, 20.61],
       [22.59, 21.35, 20.62, 20.49, 20.69],
       [22.61, 21.1 , 20.45, 20.09, 20.42]])